# Lab 3.6.4: Multi-Agent Systems

**Module:** 3.6 - AI Agents & Agentic Systems  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐⭐ (Advanced)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand multi-agent communication patterns
- [ ] Build a 3-agent content generation team
- [ ] Implement message passing between agents
- [ ] Design agent handoff protocols
- [ ] Handle agent coordination and conflicts

---

## 📚 Prerequisites

- Completed: Lab 3.6.3 (LangGraph Workflow)
- Knowledge of: Agent patterns, state management
- Running: Ollama with `llama3.1:8b` or larger model

---

## 🌍 Real-World Context

**The Problem:** Complex tasks often need multiple perspectives:
- A single agent can't be expert in everything
- Different roles require different prompts/behaviors
- Quality improves with specialization and review

**Multi-Agent Systems solve this** by having specialized agents collaborate:
- **Researcher**: Finds and gathers information
- **Writer**: Creates content from research
- **Editor**: Reviews and improves the content

**Real-World Applications:**
- **Content Creation**: Research → Write → Edit → Publish
- **Code Review**: Developer → Reviewer → Merger
- **Customer Service**: Triage → Specialist → Supervisor
- **Data Analysis**: Collector → Analyst → Visualizer → Interpreter

---

## 🧒 ELI5: What are Multi-Agent Systems?

> **Imagine a restaurant kitchen...** 👨‍🍳
>
> You don't have ONE chef doing everything. Instead:
> - **Prep Cook**: Chops vegetables, prepares ingredients
> - **Line Cook**: Actually cooks the dishes
> - **Head Chef**: Inspects plates before they go out
> - **Expeditor**: Coordinates timing and delivery
>
> Each person is specialized. They pass work to each other.
> The result? Better food, faster service!
>
> **Multi-Agent AI works the same way:**
> - Researcher Agent → gathers information
> - Writer Agent → creates the first draft
> - Editor Agent → polishes and improves
> - Each agent is specialized with different prompts and skills

```
┌─────────────────────────────────────────────────────────────────┐
│                    Multi-Agent Content Team                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   ┌───────────────┐   "Here's what I found"   ┌──────────────┐ │
│   │  🔍 RESEARCHER │ ──────────────────────► │  ✍️  WRITER   │ │
│   │               │                          │               │ │
│   │ - Searches    │                          │ - Drafts      │ │
│   │ - Summarizes  │                          │ - Structures  │ │
│   │ - Validates   │                          │ - Explains    │ │
│   └───────────────┘                          └───────┬───────┘ │
│                                                      │         │
│                                    "Here's my draft" │         │
│                                                      ▼         │
│   ┌───────────────┐   "Final version"        ┌──────────────┐ │
│   │  📤 OUTPUT    │ ◄────────────────────── │  📝 EDITOR    │ │
│   │               │                          │               │ │
│   │ - Formatted   │                          │ - Reviews     │ │
│   │ - Ready       │                          │ - Improves    │ │
│   │ - Delivered   │                          │ - Polishes    │ │
│   └───────────────┘                          └───────────────┘ │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment Setup

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
import json

# Add scripts to path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

print("✅ Standard imports successful!")

In [ ]:
# LangChain imports
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate

# Initialize LLM
llm = Ollama(
    model="llama3.1:8b",
    temperature=0.7,  # Slightly higher for creative content
    base_url="http://localhost:11434"
)

# Quick test
test = llm.invoke("Say 'Ready for multi-agent work!' in one sentence.")
print(f"✅ LLM ready: {test[:50]}...")

---

## Part 2: Defining Agent Roles

Each agent has a specific role with its own system prompt and capabilities.

In [ ]:
class AgentRole(Enum):
    """Available agent roles."""
    RESEARCHER = "researcher"
    WRITER = "writer"
    EDITOR = "editor"


@dataclass
class AgentMessage:
    """
    Message passed between agents.
    
    Attributes:
        sender: Role of the sending agent
        recipient: Role of the receiving agent
        content: The message content
        message_type: Type of message (task, result, feedback)
        timestamp: When the message was created
    """
    sender: AgentRole
    recipient: AgentRole
    content: str
    message_type: str = "task"  # task, result, feedback
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())
    
    def __str__(self):
        return f"[{self.sender.value} → {self.recipient.value}]: {self.content[:100]}..."


print("✅ AgentRole and AgentMessage classes defined!")

In [ ]:
@dataclass
class Agent:
    """
    A specialized AI agent with a specific role.
    
    Each agent has:
    - A role (researcher, writer, editor)
    - A system prompt defining its behavior
    - An LLM for generating responses
    """
    role: AgentRole
    name: str
    system_prompt: str
    llm: Any = None
    
    def process(self, input_content: str, context: str = "") -> str:
        """
        Process input and generate output based on the agent's role.
        
        Args:
            input_content: The main content to process
            context: Additional context (e.g., previous agent's work)
        
        Returns:
            The agent's response
        """
        prompt = f"""{self.system_prompt}

Context from previous work:
{context if context else 'None'}

Your task:
{input_content}

Your response:"""
        
        response = self.llm.invoke(prompt)
        return response.strip()
    
    def __str__(self):
        return f"Agent({self.name}, role={self.role.value})"


print("✅ Agent class defined!")

In [ ]:
# Define system prompts for each role
AGENT_PROMPTS = {
    AgentRole.RESEARCHER: """You are an expert Research Agent. Your job is to:
1. Gather comprehensive information on the given topic
2. Find key facts, statistics, and insights
3. Identify multiple perspectives
4. Organize information logically
5. Provide sources and citations when possible

Be thorough but concise. Focus on accuracy and relevance.
Format your research as bullet points with clear categories.""",

    AgentRole.WRITER: """You are an expert Writing Agent. Your job is to:
1. Transform research into clear, engaging content
2. Write in a conversational but professional tone
3. Structure content with clear sections and flow
4. Use examples and analogies to explain complex ideas
5. Keep paragraphs short and readable

Focus on clarity and engagement. Make the content accessible.
Always use the research provided - don't make things up.""",

    AgentRole.EDITOR: """You are an expert Editing Agent. Your job is to:
1. Review content for clarity and coherence
2. Fix grammar, spelling, and punctuation errors
3. Improve sentence structure and flow
4. Ensure consistent tone and style
5. Add polish while preserving the author's voice

Be constructive and thorough. Return the improved version.
Mark any significant changes with [EDITED: reason]."""
}

print("✅ Agent prompts defined!")
print(f"   Roles: {[r.value for r in AgentRole]}")

In [ ]:
# Create the agent team
researcher = Agent(
    role=AgentRole.RESEARCHER,
    name="ResearchBot",
    system_prompt=AGENT_PROMPTS[AgentRole.RESEARCHER],
    llm=llm
)

writer = Agent(
    role=AgentRole.WRITER,
    name="WriteBot",
    system_prompt=AGENT_PROMPTS[AgentRole.WRITER],
    llm=llm
)

editor = Agent(
    role=AgentRole.EDITOR,
    name="EditBot",
    system_prompt=AGENT_PROMPTS[AgentRole.EDITOR],
    llm=llm
)

team = [researcher, writer, editor]

print("✅ Agent team created!")
for agent in team:
    print(f"   {agent}")

---

## Part 3: Building the Agent Orchestrator

The orchestrator manages communication between agents and tracks the workflow.

In [ ]:
@dataclass
class WorkflowResult:
    """Results from a multi-agent workflow."""
    topic: str
    research: str
    draft: str
    final: str
    messages: List[AgentMessage]
    duration_seconds: float


class AgentOrchestrator:
    """
    Orchestrates communication between agents in a content creation pipeline.
    
    Workflow:
    1. Researcher gathers information
    2. Writer creates content from research
    3. Editor polishes the content
    """
    
    def __init__(self, researcher: Agent, writer: Agent, editor: Agent):
        self.researcher = researcher
        self.writer = writer
        self.editor = editor
        self.messages: List[AgentMessage] = []
        self.verbose = True
    
    def _log(self, message: str):
        """Log a message if verbose mode is on."""
        if self.verbose:
            print(message)
    
    def _record_message(self, sender: AgentRole, recipient: AgentRole, 
                        content: str, msg_type: str = "task"):
        """Record a message in the workflow history."""
        msg = AgentMessage(
            sender=sender,
            recipient=recipient,
            content=content,
            message_type=msg_type
        )
        self.messages.append(msg)
        return msg
    
    def run_pipeline(self, topic: str) -> WorkflowResult:
        """
        Run the full content creation pipeline.
        
        Args:
            topic: The topic to create content about
        
        Returns:
            WorkflowResult with all outputs and messages
        """
        import time
        start_time = time.time()
        
        self.messages = []  # Reset message history
        
        # === Step 1: Research ===
        self._log("\n" + "="*60)
        self._log("📚 STEP 1: Research Phase")
        self._log("="*60)
        
        research_task = f"Research the following topic thoroughly: {topic}"
        self._record_message(
            AgentRole.WRITER,  # Request from (conceptually)
            AgentRole.RESEARCHER,
            research_task,
            "task"
        )
        
        self._log(f"\n🔍 {self.researcher.name} is researching...")
        research_output = self.researcher.process(research_task)
        
        self._record_message(
            AgentRole.RESEARCHER,
            AgentRole.WRITER,
            research_output,
            "result"
        )
        
        self._log(f"\n📋 Research complete ({len(research_output)} chars)")
        if self.verbose:
            print(f"\n{research_output[:500]}...\n")
        
        # === Step 2: Writing ===
        self._log("\n" + "="*60)
        self._log("✍️  STEP 2: Writing Phase")
        self._log("="*60)
        
        writing_task = f"Write a comprehensive article about: {topic}"
        self._record_message(
            AgentRole.RESEARCHER,
            AgentRole.WRITER,
            writing_task,
            "task"
        )
        
        self._log(f"\n📝 {self.writer.name} is writing...")
        draft_output = self.writer.process(writing_task, context=research_output)
        
        self._record_message(
            AgentRole.WRITER,
            AgentRole.EDITOR,
            draft_output,
            "result"
        )
        
        self._log(f"\n📄 Draft complete ({len(draft_output)} chars)")
        if self.verbose:
            print(f"\n{draft_output[:500]}...\n")
        
        # === Step 3: Editing ===
        self._log("\n" + "="*60)
        self._log("📝 STEP 3: Editing Phase")
        self._log("="*60)
        
        editing_task = "Review and polish the following article for publication."
        self._record_message(
            AgentRole.WRITER,
            AgentRole.EDITOR,
            editing_task,
            "task"
        )
        
        self._log(f"\n✏️  {self.editor.name} is editing...")
        final_output = self.editor.process(editing_task, context=draft_output)
        
        self._record_message(
            AgentRole.EDITOR,
            AgentRole.WRITER,
            final_output,
            "result"
        )
        
        self._log(f"\n✅ Final content ready ({len(final_output)} chars)")
        
        duration = time.time() - start_time
        
        return WorkflowResult(
            topic=topic,
            research=research_output,
            draft=draft_output,
            final=final_output,
            messages=self.messages,
            duration_seconds=duration
        )


print("✅ AgentOrchestrator class defined!")

In [ ]:
# Create the orchestrator
orchestrator = AgentOrchestrator(researcher, writer, editor)

print("✅ Orchestrator ready!")
print(f"   Pipeline: Researcher → Writer → Editor")

---

## Part 4: Running the Multi-Agent Pipeline

In [ ]:
# Run the pipeline!
print("🚀 Starting Multi-Agent Content Pipeline...")
print("="*60 + "\n")

result = orchestrator.run_pipeline(
    topic="The benefits and challenges of running large language models locally on DGX Spark vs using cloud APIs"
)

print("\n" + "="*60)
print("🎉 PIPELINE COMPLETE!")
print("="*60)
print(f"\n⏱️  Duration: {result.duration_seconds:.1f} seconds")
print(f"💬 Messages exchanged: {len(result.messages)}")

In [ ]:
# View the final output
print("\n" + "="*60)
print("📤 FINAL ARTICLE")
print("="*60)
print(result.final)

In [ ]:
# View message history
print("\n" + "="*60)
print("💬 MESSAGE HISTORY")
print("="*60)

for i, msg in enumerate(result.messages, 1):
    print(f"\n{i}. [{msg.message_type.upper()}]")
    print(f"   From: {msg.sender.value}")
    print(f"   To: {msg.recipient.value}")
    print(f"   Content: {msg.content[:100]}...")

---

## Part 5: Advanced Pattern - Feedback Loop

Let's add a feedback loop where the editor can request revisions from the writer.

In [ ]:
class FeedbackOrchestrator(AgentOrchestrator):
    """
    Extended orchestrator that supports feedback loops.
    
    The editor can request revisions, and the writer will revise.
    """
    
    def __init__(self, *args, max_revisions: int = 2, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_revisions = max_revisions
    
    def _editor_review(self, content: str) -> tuple:
        """
        Have the editor review content and decide if revisions are needed.
        
        Returns:
            (needs_revision: bool, feedback: str)
        """
        review_prompt = f"""Review this content and determine if it needs revision.

Content:
{content[:2000]}

Respond in this format:
VERDICT: [APPROVED or NEEDS_REVISION]
FEEDBACK: [Your specific feedback for improvement]

Your review:"""
        
        review = self.editor.llm.invoke(review_prompt)
        
        needs_revision = "NEEDS_REVISION" in review.upper()
        
        # Extract feedback
        feedback = ""
        if "FEEDBACK:" in review:
            feedback = review.split("FEEDBACK:")[-1].strip()
        
        return needs_revision, feedback
    
    def run_pipeline_with_feedback(self, topic: str) -> WorkflowResult:
        """
        Run the pipeline with potential revision cycles.
        """
        import time
        start_time = time.time()
        self.messages = []
        
        # Step 1: Research (same as before)
        self._log("\n🔍 Researching...")
        research_task = f"Research: {topic}"
        research_output = self.researcher.process(research_task)
        
        # Step 2: Initial Draft
        self._log("\n✍️  Writing initial draft...")
        writing_task = f"Write an article about: {topic}"
        current_draft = self.writer.process(writing_task, context=research_output)
        
        # Step 3: Revision Loop
        revision_count = 0
        while revision_count < self.max_revisions:
            self._log(f"\n📋 Review cycle {revision_count + 1}...")
            
            needs_revision, feedback = self._editor_review(current_draft)
            
            if not needs_revision:
                self._log("\n✅ Editor approved!")
                break
            
            self._log(f"\n🔄 Revision requested: {feedback[:100]}...")
            
            # Record feedback message
            self._record_message(
                AgentRole.EDITOR,
                AgentRole.WRITER,
                feedback,
                "feedback"
            )
            
            # Writer revises
            revision_task = f"""Revise this content based on editor feedback.
            
Feedback: {feedback}

Original content:
{current_draft}

Revised content:"""
            
            current_draft = self.writer.process(revision_task)
            revision_count += 1
        
        # Final polish
        self._log("\n✏️  Final editing...")
        final_output = self.editor.process(
            "Polish this article for publication.", 
            context=current_draft
        )
        
        duration = time.time() - start_time
        
        self._log(f"\n🎉 Complete! ({revision_count} revisions)")
        
        return WorkflowResult(
            topic=topic,
            research=research_output,
            draft=current_draft,
            final=final_output,
            messages=self.messages,
            duration_seconds=duration
        )


print("✅ FeedbackOrchestrator defined!")

In [ ]:
# Test the feedback loop
feedback_orchestrator = FeedbackOrchestrator(
    researcher, writer, editor,
    max_revisions=2
)

print("🔄 Running pipeline with feedback loop...\n")

result2 = feedback_orchestrator.run_pipeline_with_feedback(
    topic="Best practices for prompt engineering"
)

print(f"\n⏱️  Duration: {result2.duration_seconds:.1f}s")
print(f"💬 Messages: {len(result2.messages)}")

---

## Part 6: Pattern - Supervisor Agent

Sometimes you need a supervisor agent that decides which agent should handle each task.

In [ ]:
class SupervisorAgent:
    """
    A supervisor agent that routes tasks to specialized agents.
    """
    
    def __init__(self, llm, agents: Dict[str, Agent]):
        self.llm = llm
        self.agents = agents
        
    def route_task(self, task: str) -> str:
        """
        Determine which agent should handle a task.
        
        Args:
            task: The task description
            
        Returns:
            Name of the agent to handle the task
        """
        agent_list = "\n".join([
            f"- {name}: {agent.system_prompt[:100]}..."
            for name, agent in self.agents.items()
        ])
        
        prompt = f"""You are a supervisor. Route this task to the best agent.

Available agents:
{agent_list}

Task: {task}

Respond with ONLY the agent name (e.g., "researcher" or "writer"):"""
        
        response = self.llm.invoke(prompt).strip().lower()
        
        # Find matching agent
        for name in self.agents.keys():
            if name.lower() in response:
                return name
        
        # Default to first agent
        return list(self.agents.keys())[0]
    
    def delegate(self, task: str) -> tuple:
        """
        Route and execute a task.
        
        Returns:
            (agent_name, result)
        """
        agent_name = self.route_task(task)
        agent = self.agents[agent_name]
        
        print(f"📋 Routing to: {agent_name}")
        result = agent.process(task)
        
        return agent_name, result


# Create supervisor
supervisor = SupervisorAgent(
    llm=llm,
    agents={
        "researcher": researcher,
        "writer": writer,
        "editor": editor
    }
)

print("✅ SupervisorAgent created!")

In [ ]:
# Test the supervisor
print("🧪 Testing Supervisor Agent:")

test_tasks = [
    "Find information about the latest GPT-4 capabilities",
    "Write a blog post about machine learning",
    "Review this article for grammar and clarity"
]

for task in test_tasks:
    print(f"\n📥 Task: {task[:50]}...")
    agent_name, result = supervisor.delegate(task)
    print(f"   Result preview: {result[:100]}...")

---

## ⚠️ Common Mistakes

### Mistake 1: Agents Not Sharing Context

In [ ]:
print("""
❌ WRONG: Each agent works in isolation

research = researcher.process("Research AI")
draft = writer.process("Write about AI")  # No context from research!

✅ RIGHT: Pass context between agents

research = researcher.process("Research AI")
draft = writer.process("Write about AI", context=research)  # Research informs writing!
""")

### Mistake 2: Infinite Feedback Loops

In [ ]:
print("""
❌ WRONG: No limit on revisions

while True:  # Could loop forever!
    needs_revision, feedback = editor.review(draft)
    if needs_revision:
        draft = writer.revise(draft, feedback)

✅ RIGHT: Set a maximum revision count

max_revisions = 3
for i in range(max_revisions):
    needs_revision, feedback = editor.review(draft)
    if not needs_revision:
        break
    draft = writer.revise(draft, feedback)
""")

---

## 🎉 Checkpoint

You've learned:
- ✅ Defining specialized agent roles
- ✅ Message passing between agents
- ✅ Building a content creation pipeline
- ✅ Implementing feedback loops
- ✅ Creating a supervisor agent for routing

---

## ✋ Try It Yourself

1. Add a **Fact-Checker** agent that verifies the writer's claims
2. Create a **Parallel Review** where multiple editors review simultaneously
3. Implement **Agent Voting** where agents vote on the best version

---

## 🚀 Challenge (Optional)

Build a **Code Review Team**:
- Developer Agent: Writes code based on requirements
- Reviewer Agent: Reviews for bugs and best practices
- Security Agent: Checks for security vulnerabilities
- Documentation Agent: Adds comments and docs

---

## 📖 Further Reading

- [AutoGen Multi-Agent Framework](https://microsoft.github.io/autogen/)
- [CrewAI for Role-Based Agents](https://docs.crewai.com/)
- [LangGraph Multi-Agent Patterns](https://langchain-ai.github.io/langgraph/)

---

## 🧹 Cleanup

In [ ]:
import gc
gc.collect()

try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
except:
    pass

print("✅ Cleanup complete!")

---

## 🎓 Summary

| Pattern | Description | Use Case |
|---------|-------------|----------|
| **Sequential Pipeline** | A → B → C | Content creation |
| **Feedback Loop** | A → B → (review) → A | Quality improvement |
| **Supervisor Routing** | S → [A, B, or C] | Dynamic task assignment |
| **Parallel Processing** | [A, B, C] → Combine | Multi-perspective analysis |

**Key Takeaways:**
- Each agent has a specialized role and prompt
- Context must be passed between agents
- Always set limits on loops/iterations
- Record messages for debugging/auditing

**Next up:** Lab 3.6.5 - CrewAI for Production Multi-Agent Systems